In [108]:
import requests
import pandas as pd

# Get data
alerts_url = 'https://www.oref.org.il//Shared/Ajax/GetAlarmsHistory.aspx?lang=he&fromDate=09.05.2021&toDate=20.05.2021&mode=0'
alerts_json = requests.get(alerts_url).json()

# Break multi-region alerts into separate records
df = pd.DataFrame.from_records(alerts_json)
df['data'] = df['data'].str.split(',')
df = df.explode('data')

# Remove sub-regions such as א, ב, ג, ד
df = df[df['data'].str.len() > 2]

# Change Hatzor to detailed name as the google geocoder fail to detect the correct city
df['data'] = df['data'].replace('חצור', 'חצור אשדוד')

# Map city names to coordinates
def get_coordinates(city_name):
    city_name = city_name + ', ישראל'
    api_key = 'YOUR_API_KEY'
    geocoder_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={city_name}&key={api_key}&language=iw'
    geocoding_result = requests.get(geocoder_url).json()
    lat = geocoding_result['results'][0]['geometry']['location']['lat']
    long = geocoding_result['results'][0]['geometry']['location']['lng']
    return(lat, long)
    
city_to_coords = {}
for city in df['data'].unique():
    try:
        city_to_coords[city] = get_coordinates(city)
        print(city, '\t -', city_to_coords[city])
    except Exception as e:
        city_to_coords[city] = (None, None)
        print(city, '\t', '- failed finding coordinates')

# Apply mapping on all data
df['outLat'] = df['data'].apply(lambda x: city_to_coords[x][0])
df['outLong'] = df['data'].apply(lambda x: city_to_coords[x][1])

# Fixed Gaza coordinates
df['inLat'] = 31.513
df['inLong'] = 34.452

# Filter wrong coordinates outside of Israel polygon (only if you use a bad geocoder)
'''
filtered_df = df[(df['outLong'] < 35.8)
                & (df['outLong'] > 33.3)
                & (df['outLat'] < 34.0)
                & (df['outLat'] > 29.2)]

filtered_df.to_csv('RocketLaunchData - Filtered.csv', encoding='utf-8-sig', index=False)
display(filtered_df)
'''

df.to_csv('RocketLaunchData.csv', encoding='utf-8-sig', index=False)
display(df)

,data,date,time,datetime,outLat,outLong,inLat,inLong
0,נחל עוז,14.05.2021,21:15,2021-05-14T21:16:00,31.472680,34.497675,31.513,34.452
1,ניר עוז,14.05.2021,21:10,2021-05-14T21:11:00,31.309697,34.402075,31.513,34.452
2,ניצנים,14.05.2021,21:10,2021-05-14T21:10:00,31.717845,34.634636,31.513,34.452
3,אשקלון,14.05.2021,21:09,2021-05-14T21:10:00,31.668789,34.574252,31.513,34.452
4,אשדוד - ח,14.05.2021,21:09,2021-05-14T21:10:00,31.804381,34.655314,31.513,34.452
...,...,...,...,...,...,...,...,...
4064,נתיב העשרה,09.05.2021,21:11,2021-05-09T21:11:00,31.572407,34.539322,31.513,34.452
4065,כרמיה,09.05.2021,21:11,2021-05-09T21:11:00,31.602695,34.540413,31.513,34.452
4066,יד מרדכי,09.05.2021,21:11,2021-05-09T21:11:00,31.586963,34.557500,31.513,34.452
4067,אזור תעשייה הדרומי אשקלון,09.05.2021,21:11,2021-05-09T21:11:00,31.636089,34.554535,31.513,34.452
